# Kafka Producer: dealer #1

In [ ]:
pip install confluent_kafka --quiet

In [ ]:
from confluent_kafka import Producer, KafkaError
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
import string

topic_cars = "DEALER1_CARS"
topic_trucks = "DEALER1_TRUCKS"

a = AdminClient({'bootstrap.servers': 'kafka:9092'})
fs = a.create_topics([NewTopic(topic_cars, num_partitions=1, replication_factor=1),
                      NewTopic(topic_trucks, num_partitions=1, replication_factor=1)])

for t, f in fs.items():
    try:
        f.result()  # The result itself is None
        print("Topic {} created".format(t))
    except Exception as e:
        print("Failed to create topic {}: {}".format(t, e))

Generate and publish a new topic message every 10 seconds.

In [ ]:
producerconf = {'bootstrap.servers': 'kafka:9092'}
prod = Producer(producerconf)

In [ ]:
from csv import DictReader
from datetime import datetime
import time
import random
import json

cars = iter(DictReader(open("/home/jovyan/data_generator/cars.csv", 'r')))
trucks = iter(DictReader(open("/home/jovyan/data_generator/trucks.csv", 'r')))

carEnd=False
truckEnd=False


# Kafka topic writing
while carEnd==False or truckEnd==False:
    
    date_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    if random.randint(0,1) == 0: # Cars
        car = next(cars, None)
        if car is not None:
            car['ts'] = date_time
            print("CAR: {}".format(car))
            prod.produce(topic=topic_cars, value=json.dumps(car))
        else:
            carEnd=True
    else: # Trucks
        truck = next(trucks, None)
        if truck is not None:
            truck['ts'] = date_time
            print("TRUCK: {}".format(truck))
            prod.produce(topic=topic_trucks, value=json.dumps(truck))
        else:
            truckEnd=True
    
    # Random data generation sleeping interval
    time.sleep(random.randint(20,40))